In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
def send_bt(msg):
    try:
        ser.write(msg.encode())
    except Exception as e:
        print("Serial error:", e)

In [21]:
def calculate_angle(a, b):
    a = np.array(a)
    b = np.array(b)

    radians = np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.degrees(radians)
    
    if angle > 180.0:
        angle = 360 - angle

    return angle

In [4]:
leftWrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
rightWrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

leftWrist, rightWrist

([0.8359602093696594, 1.5596753358840942],
 [0.12784937024116516, 1.621458888053894])

In [ ]:
import serial, time
ser = serial.Serial('/dev/rfcomm0', 9600, timeout=1)
time.sleep(2)

cap = cv2.VideoCapture(0)

#Curl Counter Variables
counter = 0
stage = None

## setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    ret, frame = cap.read()

    # Detect stuff and render
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    results = pose.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    h, w, _ = image.shape
    
      

    # Extract landmarks
    try:
        landmarks = results.pose_landmarks.landmark

        # get coordinates
        leftWrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        rightWrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

        # calculate angle
        angle = calculate_angle(leftWrist, rightWrist)

        # Hand Rotation Logic
        if angle > 30 and stage != 'left':
            stage = 'left'
            send_bt("L\n")
        elif angle < -30 and stage != 'right':
            stage = 'right'
            send_bt("R\n")
        elif -10 <= angle <= 10:  # Neutral zone
            stage = None
            
        # print(landmarks)
    except:
        pass

    lw = tuple(np.multiply(leftWrist, [w, h]).astype(int))
    rw = tuple(np.multiply(rightWrist, [w, h]).astype(int))

    cv2.line(image, lw, rw, (0, 255, 0), 3)
    cv2.line(image, (0, 450), (w, 450), (0, 255, 0), 2)

    cv2.circle(image, lw, 6, (0, 0, 255), -1)
    cv2.circle(image, rw, 6, (0, 0, 255), -1)
    
    cv2.putText(image, "LW", lw, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
    cv2.putText(image, "RW", rw, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    # Render curl counter
    # setup status box
    cv2.rectangle(image, (0,0), (500, 73), (245, 117, 16), -1)

    # Rep data
    cv2.putText(image, 
                'Rotation',
                (15, 12),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA
               )
    cv2.putText(image, 
                str(angle),
                (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA
               )

    # Stage data
    cv2.putText(image, 
                'STAGE -> ',
                (95, 12),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA
               )
    cv2.putText(image, 
                stage,
                (150, 18),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA
               )
      

    #Render detections
    # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
    #                          mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
    #                          mp_drawing.DrawingSpec(color=(245, 66, 200), thickness=2, circle_radius=2)
    #                          )

    # print(results)

    cv2.imshow('Mediapipe Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

ser.close()
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1765954098.810179    3221 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1765954098.811613    9872 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.3.1-arch1.2), renderer: AMD Radeon 780M Graphics (radeonsi, phoenix, LLVM 21.1.6, DRM 3.64, 6.17.9-arch1-1)
W0000 00:00:1765954098.833667    9854 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765954098.845336    9861 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [15]:
cv2.putText?

Docstring:
putText(img, text, org, fontFace, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]]) -> img
.   @brief Draws a text string.
.   
.   The function cv::putText renders the specified text string in the image. Symbols that cannot be rendered
.   using the specified font are replaced by question marks. See #getTextSize for a text rendering code
.   example.
.   
.   @param img Image.
.   @param text Text string to be drawn.
.   @param org Bottom-left corner of the text string in the image.
.   @param fontFace Font type, see #HersheyFonts.
.   @param fontScale Font scale factor that is multiplied by the font-specific base size.
.   @param color Text color.
.   @param thickness Thickness of the lines used to draw a text.
.   @param lineType Line type. See #LineTypes
.   @param bottomLeftOrigin When true, the image data origin is at the bottom-left corner. Otherwise,
.   it is at the top-left corner.
Type:      builtin_function_or_method

In [24]:
cv2.rectangle?


Docstring:
rectangle(img, pt1, pt2, color[, thickness[, lineType[, shift]]]) -> img
.   @brief Draws a simple, thick, or filled up-right rectangle.
.   
.   The function cv::rectangle draws a rectangle outline or a filled rectangle whose two opposite corners
.   are pt1 and pt2.
.   
.   @param img Image.
.   @param pt1 Vertex of the rectangle.
.   @param pt2 Vertex of the rectangle opposite to pt1 .
.   @param color Rectangle color or brightness (grayscale image).
.   @param thickness Thickness of lines that make up the rectangle. Negative values, like #FILLED,
.   mean that the function has to draw a filled rectangle.
.   @param lineType Type of the line. See #LineTypes
.   @param shift Number of fractional bits in the point coordinates.



rectangle(img, rec, color[, thickness[, lineType[, shift]]]) -> img
.   @overload
.   
.   use `rec` parameter as alternative specification of the drawn rectangle: `r.tl() and
.   r.br()-Point(1,1)` are opposite corners
Type:      builtin_function